#### predict house prices

In [650]:
import pandas as pd

In [651]:
File_path = "data/House_Rent_Dataset.csv"

df = pd.read_csv(File_path)

In [652]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


#### Data preprocessing

In [653]:
df.describe()

,BHK,Rent,Size,Bathroom
count,4746.000000,4.746000e+03,4746.000000,4746.000000
mean,2.083860,3.499345e+04,967.490729,1.965866
std,0.832256,7.810641e+04,634.202328,0.884532
min,1.000000,1.200000e+03,10.000000,1.000000
25%,2.000000,1.000000e+04,550.000000,1.000000
50%,2.000000,1.600000e+04,850.000000,2.000000
75%,3.000000,3.300000e+04,1200.000000,2.000000
max,6.000000,3.500000e+06,8000.000000,10.000000


In [654]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [655]:
df.isnull().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

In [656]:
df.duplicated().sum()

0

In [657]:
df['City'].unique()

array(['Kolkata', 'Mumbai', 'Bangalore', 'Delhi', 'Chennai', 'Hyderabad'],
      dtype=object)

In [658]:
# Dropping columns in-place
df.drop(['Posted On', 'Point of Contact'], axis=1, inplace=True)

df.head()


,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom
0,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2
1,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1
2,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1
3,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1
4,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1


#### Check for outliers

In [659]:
import numpy as np
from scipy import stats

# Calculate Z-scores of 'Rent'
z_scores = np.abs(stats.zscore(df['Rent']))
# np.where returns a tuple with arrays in it, you need the first element of the tuple which is the array of indices
outliers_z = np.where(z_scores > 3)[0]  # Note the [0] to select the array from the tuple

# Calculate IQR
Q1 = df['Rent'].quantile(0.25)
Q3 = df['Rent'].quantile(0.75)
IQR = Q3 - Q1
# Directly use boolean indexing for IQR outliers
outliers_iqr = df[(df['Rent'] < (Q1 - 1.5 * IQR)) | (df['Rent'] > (Q3 + 1.5 * IQR))]

# Print results
print("Outliers using Z-score:", df['Rent'].iloc[outliers_z])  # Use iloc for index-based indexing


Outliers using Z-score: 531     300000
542     270000
543     400000
579     300000
589     350000
         ...  
3148    330000
3320    280000
3656    600000
3989    300000
4457    400000
Name: Rent, Length: 66, dtype: int64


In [660]:
# Remove outliers based on Z-score
df_clean_z = df.drop(index=outliers_z)

# Remove outliers based on IQR
df_clean_iqr = df.drop(index=outliers_iqr.index)


In [661]:
df_clean_iqr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4226 entries, 0 to 4745
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4226 non-null   int64 
 1   Rent               4226 non-null   int64 
 2   Size               4226 non-null   int64 
 3   Floor              4226 non-null   object
 4   Area Type          4226 non-null   object
 5   Area Locality      4226 non-null   object
 6   City               4226 non-null   object
 7   Furnishing Status  4226 non-null   object
 8   Tenant Preferred   4226 non-null   object
 9   Bathroom           4226 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 363.2+ KB


In [662]:
df_clean_z.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4680 entries, 0 to 4745
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4680 non-null   int64 
 1   Rent               4680 non-null   int64 
 2   Size               4680 non-null   int64 
 3   Floor              4680 non-null   object
 4   Area Type          4680 non-null   object
 5   Area Locality      4680 non-null   object
 6   City               4680 non-null   object
 7   Furnishing Status  4680 non-null   object
 8   Tenant Preferred   4680 non-null   object
 9   Bathroom           4680 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 402.2+ KB


In [663]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4746 non-null   int64 
 1   Rent               4746 non-null   int64 
 2   Size               4746 non-null   int64 
 3   Floor              4746 non-null   object
 4   Area Type          4746 non-null   object
 5   Area Locality      4746 non-null   object
 6   City               4746 non-null   object
 7   Furnishing Status  4746 non-null   object
 8   Tenant Preferred   4746 non-null   object
 9   Bathroom           4746 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 370.9+ KB


In [664]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4746 non-null   int64 
 1   Rent               4746 non-null   int64 
 2   Size               4746 non-null   int64 
 3   Floor              4746 non-null   object
 4   Area Type          4746 non-null   object
 5   Area Locality      4746 non-null   object
 6   City               4746 non-null   object
 7   Furnishing Status  4746 non-null   object
 8   Tenant Preferred   4746 non-null   object
 9   Bathroom           4746 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 370.9+ KB


#### one hot encoding

In [665]:
df_clean_iqr = pd.get_dummies(df_clean_iqr, columns=['Furnishing Status', 'Area Type', 'City'])

df_clean_iqr.head()

,BHK,Rent,Size,Floor,Area Locality,Tenant Preferred,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai
0,2,10000,1100,Ground out of 2,Bandel,Bachelors/Family,2,False,False,True,False,False,True,False,False,False,False,True,False
1,2,20000,800,1 out of 3,"Phool Bagan, Kankurgachi",Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False
2,2,17000,1000,1 out of 3,Salt Lake City Sector 2,Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False
3,2,10000,800,1 out of 2,Dumdum Park,Bachelors/Family,1,False,False,True,False,False,True,False,False,False,False,True,False
4,2,7500,850,1 out of 2,South Dum Dum,Bachelors,1,False,False,True,False,True,False,False,False,False,False,True,False


In [666]:
df_clean_iqr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4226 entries, 0 to 4745
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   BHK                               4226 non-null   int64 
 1   Rent                              4226 non-null   int64 
 2   Size                              4226 non-null   int64 
 3   Floor                             4226 non-null   object
 4   Area Locality                     4226 non-null   object
 5   Tenant Preferred                  4226 non-null   object
 6   Bathroom                          4226 non-null   int64 
 7   Furnishing Status_Furnished       4226 non-null   bool  
 8   Furnishing Status_Semi-Furnished  4226 non-null   bool  
 9   Furnishing Status_Unfurnished     4226 non-null   bool  
 10  Area Type_Built Area              4226 non-null   bool  
 11  Area Type_Carpet Area             4226 non-null   bool  
 12  Area Type_Super Area     

In [667]:
df_clean_iqr['Area Locality'].nunique()

1997

In [668]:
df_clean_iqr['Floor'].unique()

array(['Ground out of 2', '1 out of 3', '1 out of 2', 'Ground out of 1',
       'Ground out of 4', '1 out of 4', '1 out of 1', 'Ground out of 3',
       '2 out of 3', '4 out of 5', '2 out of 2', '2 out of 5',
       '4 out of 14', '3 out of 3', '5 out of 5', '4 out of 4',
       '7 out of 8', '2 out of 4', '3 out of 4', '1 out of 5',
       '8 out of 5', 'Ground out of 6', '2 out of 1',
       'Upper Basement out of 4', 'Ground out of 5', '3 out of 5',
       '11 out of 19', '5 out of 10', '11 out of 14',
       'Lower Basement out of 2', '2 out of 7', '7 out of 10',
       '6 out of 7', '4 out of 7', '2 out of 8', '5 out of 12',
       '11 out of 21', '14 out of 23', '9 out of 20',
       'Upper Basement out of 9', '19 out of 24', '3 out of 21',
       '1 out of 22', '3 out of 7', '8 out of 8', '6 out of 12',
       'Upper Basement out of 16', '34 out of 48', '5 out of 8',
       '5 out of 14', '14 out of 40', '9 out of 22', '12 out of 18',
       '26 out of 44', '1 out of 8', 'Ground

#### Extract Features Current_Floor and Total_Floors from Floor feature

In [669]:
def parse_floor(value):
    if 'out of' in value:
        current_floor, total_floors = value.split(' out of ')
        # Replace special floor names with numbers
        floor_mapping = {
            'Ground': 0,
            'Upper Basement': -1,
            'Lower Basement': -2
        }
        current_floor = floor_mapping.get(current_floor, current_floor)
        return int(current_floor), int(total_floors)
    return None, None

# Applying the function to each value in the Floor column
df_clean_iqr['Current_Floor'], df_clean_iqr['Total_Floors'] = zip(*df_clean_iqr['Floor'].map(parse_floor))

# Dropping the original 'Floor' column
df_clean_iqr.drop('Floor', axis=1, inplace=True)

df_clean_iqr.head()

,BHK,Rent,Size,Area Locality,Tenant Preferred,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Current_Floor,Total_Floors
0,2,10000,1100,Bandel,Bachelors/Family,2,False,False,True,False,False,True,False,False,False,False,True,False,0.0,2.0
1,2,20000,800,"Phool Bagan, Kankurgachi",Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False,1.0,3.0
2,2,17000,1000,Salt Lake City Sector 2,Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False,1.0,3.0
3,2,10000,800,Dumdum Park,Bachelors/Family,1,False,False,True,False,False,True,False,False,False,False,True,False,1.0,2.0
4,2,7500,850,South Dum Dum,Bachelors,1,False,False,True,False,True,False,False,False,False,False,True,False,1.0,2.0


In [670]:
df_clean_iqr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4226 entries, 0 to 4745
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BHK                               4226 non-null   int64  
 1   Rent                              4226 non-null   int64  
 2   Size                              4226 non-null   int64  
 3   Area Locality                     4226 non-null   object 
 4   Tenant Preferred                  4226 non-null   object 
 5   Bathroom                          4226 non-null   int64  
 6   Furnishing Status_Furnished       4226 non-null   bool   
 7   Furnishing Status_Semi-Furnished  4226 non-null   bool   
 8   Furnishing Status_Unfurnished     4226 non-null   bool   
 9   Area Type_Built Area              4226 non-null   bool   
 10  Area Type_Carpet Area             4226 non-null   bool   
 11  Area Type_Super Area              4226 non-null   bool   
 12  City_Bangal

In [671]:
df_clean_iqr[['Current_Floor', 'Total_Floors']].isna().sum()

Current_Floor    4
Total_Floors     4
dtype: int64

In [672]:
df_clean_iqr.dropna(subset=['Current_Floor', 'Total_Floors'], inplace=True)

In [673]:
df_clean_iqr.isnull().sum()

BHK                                 0
Rent                                0
Size                                0
Area Locality                       0
Tenant Preferred                    0
Bathroom                            0
Furnishing Status_Furnished         0
Furnishing Status_Semi-Furnished    0
Furnishing Status_Unfurnished       0
Area Type_Built Area                0
Area Type_Carpet Area               0
Area Type_Super Area                0
City_Bangalore                      0
City_Chennai                        0
City_Delhi                          0
City_Hyderabad                      0
City_Kolkata                        0
City_Mumbai                         0
Current_Floor                       0
Total_Floors                        0
dtype: int64

In [674]:
df_clean_iqr['Current_Floor'] = df_clean_iqr['Current_Floor'].astype(int)
df_clean_iqr['Total_Floors'] = df_clean_iqr['Total_Floors'].astype(int)

df_clean_iqr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4222 entries, 0 to 4745
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   BHK                               4222 non-null   int64 
 1   Rent                              4222 non-null   int64 
 2   Size                              4222 non-null   int64 
 3   Area Locality                     4222 non-null   object
 4   Tenant Preferred                  4222 non-null   object
 5   Bathroom                          4222 non-null   int64 
 6   Furnishing Status_Furnished       4222 non-null   bool  
 7   Furnishing Status_Semi-Furnished  4222 non-null   bool  
 8   Furnishing Status_Unfurnished     4222 non-null   bool  
 9   Area Type_Built Area              4222 non-null   bool  
 10  Area Type_Carpet Area             4222 non-null   bool  
 11  Area Type_Super Area              4222 non-null   bool  
 12  City_Bangalore           

In [675]:
df_clean_iqr.head()

,BHK,Rent,Size,Area Locality,Tenant Preferred,Bathroom,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Current_Floor,Total_Floors
0,2,10000,1100,Bandel,Bachelors/Family,2,False,False,True,False,False,True,False,False,False,False,True,False,0,2
1,2,20000,800,"Phool Bagan, Kankurgachi",Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False,1,3
2,2,17000,1000,Salt Lake City Sector 2,Bachelors/Family,1,False,True,False,False,False,True,False,False,False,False,True,False,1,3
3,2,10000,800,Dumdum Park,Bachelors/Family,1,False,False,True,False,False,True,False,False,False,False,True,False,1,2
4,2,7500,850,South Dum Dum,Bachelors,1,False,False,True,False,True,False,False,False,False,False,True,False,1,2


#### one hot encoding for coloumn Tenant Preferred

In [676]:
df_clean_iqr = pd.get_dummies(df_clean_iqr, columns=['Tenant Preferred'])

df_clean_iqr.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 4222 entries, 0 to 4745
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   BHK                                4222 non-null   int64 
 1   Rent                               4222 non-null   int64 
 2   Size                               4222 non-null   int64 
 3   Area Locality                      4222 non-null   object
 4   Bathroom                           4222 non-null   int64 
 5   Furnishing Status_Furnished        4222 non-null   bool  
 6   Furnishing Status_Semi-Furnished   4222 non-null   bool  
 7   Furnishing Status_Unfurnished      4222 non-null   bool  
 8   Area Type_Built Area               4222 non-null   bool  
 9   Area Type_Carpet Area              4222 non-null   bool  
 10  Area Type_Super Area               4222 non-null   bool  
 11  City_Bangalore                     4222 non-null   bool  
 12  City_Chenna

In [677]:

# Let's assume your DataFrame is named df
X = df_clean_iqr.drop('Rent', axis=1)  # Features
y = df_clean_iqr['Rent']  # Target



In [678]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4222 entries, 0 to 4745
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   BHK                                4222 non-null   int64 
 1   Size                               4222 non-null   int64 
 2   Area Locality                      4222 non-null   object
 3   Bathroom                           4222 non-null   int64 
 4   Furnishing Status_Furnished        4222 non-null   bool  
 5   Furnishing Status_Semi-Furnished   4222 non-null   bool  
 6   Furnishing Status_Unfurnished      4222 non-null   bool  
 7   Area Type_Built Area               4222 non-null   bool  
 8   Area Type_Carpet Area              4222 non-null   bool  
 9   Area Type_Super Area               4222 non-null   bool  
 10  City_Bangalore                     4222 non-null   bool  
 11  City_Chennai                       4222 non-null   bool  
 12  City_Delhi 

In [679]:
y.info()

<class 'pandas.core.series.Series'>
Index: 4222 entries, 0 to 4745
Series name: Rent
Non-Null Count  Dtype
--------------  -----
4222 non-null   int64
dtypes: int64(1)
memory usage: 66.0 KB


In [680]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [681]:
import category_encoders as ce

# Create a target encoder object
encoder = ce.TargetEncoder(cols=['Area Locality'])

# Fit and transform to the train data
X_train['Area Locality'] = encoder.fit_transform(X_train['Area Locality'], y_train)

# Transform on test data
X_test['Area Locality'] = encoder.transform(X_test['Area Locality'])




In [682]:
from sklearn.linear_model import LinearRegression

# Create a model instance
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [683]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2)


MSE: 65296382.37118921
RMSE: 8080.6176973786605
R^2: 0.647623350707625
